In [12]:
import copy
import json
import os


def calculate_result_dict_size(result_dict: dict) -> int:
    size: int = 0
    for bitvector in result_dict.keys():
        project_list: dict = result_dict[bitvector]
        for project_name in project_list.keys():
            size += len(project_list[project_name])

    return size


def get_exclusive_result(result_dict1: dict, result_dict2: dict) -> dict:
    result_dict = copy.deepcopy(result_dict1)
    for bitvector in result_dict1.keys():
        project_list1: dict = result_dict1[bitvector]

        if bitvector[-1] == "1":
            bitvector_2 = bitvector[:-1] + "0"
        else:
            bitvector_2 = bitvector[:-1] + "1"

        project_list2: dict = result_dict2[bitvector_2]

        for project_name in project_list1.keys():
            bid_1 = project_list1[project_name]

            if project_name in project_list2.keys():
                bid_2 = project_list2[project_name]
            else:
                bid_2 = []

            exclusive_bid = [bug_id for bug_id in bid_1 if bug_id not in bid_2]
            if len(exclusive_bid) > 0:
                result_dict[bitvector][project_name] = exclusive_bid
            else:
                del result_dict[bitvector][project_name]

        if len(result_dict[bitvector]) == 0:
            del result_dict[bitvector]

    return result_dict


def bitvector_flatten(bitvector: dict):
    bitvector_str = [str(value) for value in bitvector.values()]

    # Combine all strings into one string
    bitvector_str = "".join(bitvector_str)

    return bitvector_str


database_path = "../../LPPR-data/315-dataset-12-2023"

results_with_cot = {}
results_without_cot = {}

# for dataset in os.listdir(database_path):
#     dataset_path = os.path.join(database_path, dataset)
#     if not os.path.isdir(dataset_path):
#         continue

dataset_path = database_path
a = 0
for project in os.listdir(dataset_path):
    project_path = os.path.join(dataset_path, project)
    if not os.path.isdir(project_path):
        continue

    for bid in os.listdir(project_path):
        bid_path = os.path.join(project_path, bid)
        if not os.path.isdir(bid_path):
            continue

        for file in os.listdir(bid_path):
            file_path = os.path.join(bid_path, file)
            if "result" not in file_path:
                continue

            response_path = file_path.replace("result", "response")

            with open(response_path, "r") as response_file:
                available_strata = json.load(response_file)[project][0]["available_strata"]
            bitvector: str = bitvector_flatten(available_strata)

            with open(file_path, "r") as result_file:
                validation_result = json.load(result_file)[f"{project}:{bid}"]
            if validation_result != 0:
                continue

            if bitvector[-1] == "1":
                if bitvector not in results_with_cot.keys():
                    results_with_cot[bitvector] = {}

                if project not in results_with_cot[bitvector].keys():
                    results_with_cot[bitvector][project] = set()

                results_with_cot[bitvector][project].add(int(bid))

            else:
                if bitvector not in results_without_cot.keys():
                    results_without_cot[bitvector] = {}

                if project not in results_without_cot[bitvector].keys():
                    results_without_cot[bitvector][project] = set()

                results_without_cot[bitvector][project].add(int(bid))

print(f"{str(calculate_result_dict_size(results_with_cot))} bug-bitvector pairs are fixed with cot")

print(f"{str(calculate_result_dict_size(results_without_cot))} bug-bitvector pairs are fixed without cot")

exclusive_results_with_cot = get_exclusive_result(results_with_cot, results_without_cot)
exclusive_results_without_cot = get_exclusive_result(results_without_cot, results_with_cot)

print(f"{str(calculate_result_dict_size(exclusive_results_with_cot))} bug-bitvector pairs are fixed exclusively with cot")

print(f"{str(calculate_result_dict_size(exclusive_results_without_cot))} bug-bitvector pairs are fixed exclusively without cot")


2042 bug-bitvector pairs are fixed with cot
1536 bug-bitvector pairs are fixed without cot
869 bug-bitvector pairs are fixed exclusively with cot
363 bug-bitvector pairs are fixed exclusively without cot
